## 类Translator(object):
### 变量
- opt: 为OpenNMT-py/translate.py中的超参数
- tt: = torch.cuda if opt.cuda else torch
- beam_accum: 词典类，项目包括：predicted_ids, beam_parent_ids, scores, log_probs。分别都对应着一个列表类
- model_opt: = checkpoint['opt']
- src_dict: = checkpoint['dicts']['src']
- tgt_dict: = checkpoint['dicts']['tgt']
- _type: 模型中encoder的类型，如果模型中没有设定，则返回‘text’
- self_model: 用于翻译的模型 参数从checkpoint中获得

### 方法
##### __init__(self, opt):
- 读取checkpoint: = torch.load(opt.model, map_location = lambda storage, loc: storage)。lambda后面的是干嘛用的表示不懂？？
- 从checkpoint中读取模型的参数model_opt = checkpoint['opt']
- 从checkpoint中读取原文和目标文的词汇表src_dict tgt_dict
- 根据model_opt中encoder的类型，设定encoder模型 = onmt.Models.Encoder(model_opt, self.src_dict)
- 根据model_opt设定decoder模型 = onmt.Models.Decoder(model_opt, self.tgt_dict) <font color=gray>应该具体看一下</font>
- 根据上面的encoder和decoder，建立MT模型model = onmt.Models.NMTModel(encoder, decoder) <font color=gray>应该具体看一下</font>
- 还需自己额外补一个generator，是一个从隐藏层到目标词汇表的线性映射，并对结果进行logSoftmax处理
- 根据checkpoint设置model的参数 model.load_state_dict(checkpoint['model'])
- 根据checkpoint设置generator的参数 generator.load_state_dict(checkpoint['model'])
- 如果有设置cuda，则开启model和generator的cuda否则则使用cpu <font color=red>这个表示不懂</font>
- 启动模型model的evaluation模式

##### initBeamAccum(self):
- 初始化beam_accum

##### _getBachSize(self, batch):
- 根据_type的类型返回batch的大小

##### buildData(self, srcBatch, goldBatch):
- 主要完成把数据集中的单词转换成对应的index。然后把数据都扔进dataset类型里面进行包装（padding，根据batchsize的大小，划分成多个Batch）。
- 注意 这里的操作要求和preprocess中的一样。
- 通过词典类把srcBatch中的单词，转换成对应的index
- 如果goldBatch也存在，则对其作相同的处理（这里的goldBatch指reference译文的意思？）
- 返回Dataset类型的实例

#### buildTargetTokens(self, pred, src, attn):
- 把生成的index序列pred，转化成单词序列（去掉最后的EOS符号）
- attn不知道是什么，被用与replace_unk的操作之中？？？？

##### translateBactch(self, srcBatch, tgtBatch):
- batch size is in different location depending on data这句话不知道是什么意思？？？
- 运行model.encoder(srcBatch)得到encStates和context
- 下面内容放到代码中进行解说

In [1]:
def trainslateBatch(self, srcBatch, tgtBatch):
    beamSize = self.opt.beam_size
    # 1) run the encoder on the src
    # 其中encoder的输入的size是：[seq_len, batch, input_size], 这里seq_len即是numWords
    # 其中context是[seq_len, batch, hidden_size * num_directions]
    # encStates是一个tupel，他包括:
    #  - h_0 (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t=seq_len
    #  - c_0 (num_layers * num_directions, batch, hidden_size): tensor containing the cell state for t=seq_len
    encStates, context = self.model.encoder(srcBatch)
    
    # 这里的srcBatch原本是dataset的输出，所以应该是(src, lengths)，下面这一步取出其中src的内容。
    srcBatch = srcBatch[0]
    batchSize = self._getBatchSize(srcBatch)
    
    # 获得RNN每层的节点数
    rnnSize = context.size(2)
    
    # 转换encState的维度为：[layers * batch * (directions * dim)]
    encStates = (self.model._fix_enc_hidden(encStates[0]), self.model_fix_enc_hidden(encStates[1]))
    
    decoder = self.model.decoder
    attentionLayer = decoder.attn
    
    # 如果类型为text，且batchSize大于1，则使用mask，原因未知？？？？？
    # 这个mask 将被用于decoder中的Attention，目的是使得attention能够忽略掉输入句子的padding的部分内容。
    # 但为什么仅在batchSize大于1的时候，才使用呢？？？
    useMasking = (self._type == 'text' and batchSize > 1)
    padMask = None
    if useMasking:
        padMask = srcBatch.data.eq(onmt.Constants.PAD).t() #标记pad的内容
    
    def mask(padMask):
        if useMasking:
            attentionLayer.appleMask(padMask)
    
    # 如果有参考译文，则进行打分
    # Tensor的new()创建一个相同类型的Tensor，这里初始每个句子的分数为0
    goldScores = context.data.new(batchSize).zero_()
    if tgtBatch is not None:
        decStates = encStates
        # 初始化一个decoder的输出，
        decOut = self.model.make_init_decoder_output(context)
        mask(padMask)
        initOutput = self.model.make_init_decoder_output(context)
        # decoder的输出使outputs, hidden, atten, 前两者同一般的rnn输出，atten是nn.softmax()的输出
        # 关于tgtBatch，应该是一个size: [numWords, batchSize]的Variable(Datasetl类里面进行了转换)
        # globalAttention的参数是input: batch x hidden_size，context: batch x seq_len x hidden_size。
        # 对应输出的attn是batch X seq_len
        # 所以decoder的输出output, hidden, attn的size应该分别是：
        # [seq_len X batch X hidden_size], [num_layers X batch X hidden_size], [batch X seq_len]
        decOut, decStates, attn = self.model.decoder(tgtBatch[:-1], decStates, context, initOutput)
        for dec_t, tgt_t in zip(decOut, tgtBatch[1:].data):
            gen_t = self.model.generator.forward(dec_t) # 为什么generator跑到model下面了，不是在这个类里面定义的吗？？？
            tgt_t = tgt_t.unsqueeze(1)
            scores = gen_t.data.gather(1, tgt_t) # http://www.cnblogs.com/YiXiaoZhou/p/6387769.html
            scores.masked_fill_(tgt_t.eq(onmt.Constants.PAD), 0)
            goldScores += scores
    
    # 使用decoder来生成目标句子，
    # 根据beamSize扩张前面的tensor，
    # 这里原来的context是[seq_len, batch, hidden_size * num_directions]
    # repeat之后的context就是[seq_len, batch * beamSize, hidden_size * num_directions]
    context = Variable(context.data.repeat(1, beamSize, 1))
    decStates = (Variable(encStates[0].data.repeat(1, beamSize, 1)),
                 Variable(encStates[1].data.repeat(1, beamSize, 1)))
    
    # 生成beamSize个onmt.Beam对象
    beam = [onmt.Beam(beamSize, self.opt.cuda) for k in range(batchSize)]
    
    decOut = self.model.make_init_decoder_output(context)
    
    if useMasking:
        padMask = srcBatch.data.eq(onmt.Constant.PAD).t().unsqueeze(0).repeat(beamSize, 1, 1)
        
    batchIdx = list(range(batchSize))
    remainingSents = batchSize
    
    for i in range(self.opt.max_sent_length):
        mask(padMask)
        # prepare decoder input？？ baem中并不包含有input的信息，这个input是怎么能够无中生有的
        input = torch.stack([b.getCurrentState() for b in beam
                             if not b.done()]).t().contiguous().view(1, -1)
        # 官方给出这里的decOut的size是[1 X (beam*batch) X numWords] 和前面的推测有点出入，稍等一下？？？？我得回去翻一下model的代码
        decOut, decStates, attn = self.model.decoder(Variable(input, volatile = True),
                                                    decStates, context, decOut)
        

##### translate(self, srcBatch, goldBatch):

In [ ]:
def translate(self, srcBatch, goldBatch):
    # 把单词转化成对应的index，然后放进dataset中进行包装
    dataset = self.buildData(srcBatch, goldBatch)
    # 获得第一个Batch
    src, tgt, indices = dataset[0]
    batchSize = self._getBatchSize(src[0])
    
    # 扔到translateBatch方法里面进行翻译, 这里src，tgt都是tensor类型维度为batchSize*numWord
    pred, predScore, attn, goldScore = self.translateBatch(src, tgt)
    # 根据indices，对上面得到的结果进行排序。使顺序回到通过lengths进行排序之前。
    pred, predScore, attn, glodScore = list(zip(*sorted(zip(pred, predScore, attn, goldScore, indices),
                                                        key = lambda x: x[-1])))[:-1]
    
    # 把indexes转换会对应的单词
    predBatch = []
    for b in range(batchSize):
        predBatch.append(
            [self.buildTargetTokens(pred[b][n], srcBatch[b], attn[b][n])
             for n in range(self.opt.n_best)]
        )
    return predBatch, predScore, goldScore